<img align="right" style="max-width: 200px; height: auto" src="./assets/logo.png">

## Lab 02 - Daten Akquise, Aufbereitung und Validierung

Lehrgang Internal Auditing, Universität St.Gallen (HSG), 2022

Die Analysen des Seminars **Lehrgangs Internal Auditing** basieren auf Jupyter Notebook (https://jupyter.org). Anhand solcher Notebooks ist es möglich eine Vielzahl von Datenanalysen und statistischen Validierungen durchzuführen. 

<img align="center" style="max-width: 700px" src="./assets/banner.png">

Im Rahmen des vorhergehenden Labs haben wir einige der grundlegenden Funktionen von Jupyter Notebook kennengelernt. In diesem Praktikum werden wir Jupyter Notebook verwenden, um ausgewählte **Schritte des im Rahmen des Seminars vorgestellten Datenanalyseprozesses** praktisch zu vertiefen. Das Hauptziel dieses Notebook ist es, die einzelne Schritte des Datenimports und der Datenvalidierung anhand eines konkreten Beispiels durchzuführen.

<img align="center" style="max-width: 800px; height: auto" src="./assets/analytics_process.png">

Im Zweifelsfall oder bei Fragen wenden Sie sich, wie immer gerne an uns via **marco (dot) schreyer (at) unisg (dot) ch**. Wir wünschen Ihnen Viel Freude mit unseren Notebooks und Ihren forensischen Analysen!

## Lernziele des Labs:

Nach der heutigen Übung sollten Sie in der Lage sein:

> 1. Erste eigene Datenakquisen mit **Jupyter** und **Python** durchzuführen.
> 2. Die Bibliotheken **Pandas** und **NumPy** zu verwenden, um eine Vielzahl von strukturierten Daten zu validieren. 
> 3. Die Bibliotheken **Matplotlib** und **Seaborn** zu verwenden, um eigene Datenvisualisierungen zu erstellen.
> 4. Erste **konkrete Ideen** für mögliche Datenanalysen in Ihrem Unternehmen zu entwickeln.

Doch bevor wir beginnen, sehen wir uns ein kurzes Motivationsvideo an, welches im Jahr 2019 durch das **American Institute of Certified Public Accountants (AICPA)** über die Anwendung von Jupyter Notebooks und die Programmiersprache Python veröffentlicht wurde: 

In [2]:
from IPython.display import YouTubeVideo
# AICPA: Upgrade the Financial Statement Audit with Audit Data Analytics
# YouTubeVideo('kHY-Ioq_InA', width=800, height=600)

## 1. Einrichtung der Jupyter Notebook-Umgebung

In Analogie zu unserem einführenden Notebook ist es zunächst wieder notwendig, einige Python-Bibliotheken zu importieren, die es uns ermöglichen, Daten zu importieren, zu analysieren und zu visualisieren. In diesem Notebook werden wir hierzu wieder die in **Lab 01** vorgestellten Bibliotheken **(1) Pandas** (https://pandas.pydata.org), **(2) NumPy** (https://numpy.org) und **(3) Matplotlib** (https://matplotlib.org) verwenden.

Lassen Sie uns nun die beiden wichtigsten Datenanalyse-Bibliotheken `Pandas` und `NumPy` entsprechend importieren, indem wir die nachfolgenden beiden `import` Anweisungen ausführen:

In [ ]:
import pandas as pd
import numpy as np

# setzen globaler Pandas Parameter
pd.options.display.max_rows = 500 # allgemeine Darstellung Anzahl Zeilen
pd.options.display.float_format = '{:.2f}'.format # numerische Darstellung von Gleitkommazahlen

Ausserdem importieren wir einige **Utility Bibliotheken**, d.h. Bibliotheken die wichtige zusätzliche Funktionalität zur Verfügung stellen:

In [ ]:
import os # ermöglicht die Erstellung, den Zugriff und die Manipulation von Datenverzeichnissen
import datetime as dt # ermöglicht die Erstellung von Zeitstempeln für Daten

Auch importieren wir eine Reihe von **zusätzlichen Bibliotheken** für den Datenzugriff und den Datenimport in Python:

In [ ]:
import io # ermöglicht das Öffnen und den Zugriff auf Datenströme
import pathlib # ermöglicht das Schreiben von lokaken Dateien
import urllib.request # ermöglicht die Erstellung von Webseiten Anfragen

Schliesslich importieren wir die Bibliotheken `Matplotlib` und `Seaborn` und setzen einige der allgemeinen Parameter für die Datenvisualisierung:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# setzen globaler Matplotlib Paramater der Datenvisualisierung
plt.style.use('seaborn') # den Visualisierungsstil festlegen
plt.rcParams['figure.figsize'] = [5, 5] # die Visualisierungsgrösse festlegen
plt.rcParams['figure.dpi']= 150 # die Visualisierungsauflösung festlegen

Die nachfolgende Anweisung aktiviert das sog. **Inline-Plotten** von Schaubildern innerhalb des aktuellen Notebooks:

In [ ]:
%matplotlib inline

Darüber hinaus ist es gute Praxis, für jedes Notebook eine entsprechende Ordnerstruktur anzulegen. Diese Ordnerstruktur dient im weiteren Analyseverlauf dazu sowohl die **Originaldaten** als auch **Analyseergebnisse** und **Schaubilder** zu speichern:

In [ ]:
# erstellen des Verzeichnis der Orignaldaten
original_data_dir = './01_original_data'
if not os.path.exists(original_data_dir): os.makedirs(original_data_dir) 
    
# erstellen des Verzeichnis der Analysedaten
analysis_data_dir = './02_analysis_data'
if not os.path.exists(analysis_data_dir): os.makedirs(analysis_data_dir)
    
# erstellen des Verzeichnis der Validierungsergebnisse
validation_data_dir = './03_validation_results'
if not os.path.exists(validation_data_dir): os.makedirs(validation_data_dir) 

Abschliessend möchten wir mögliche **Warnungen** einzelner Bibliotheken ignorieren:

In [ ]:
import warnings # ermöglicht die Handhabung von Warnmeldungen

# setzen des Warnfilter-Flags, um Warnungen zu ignorieren
warnings.filterwarnings('ignore')

Solche Warnungen können oftmals aufgrund von aktuellen Bibliothekserweiterungen bzw. -weiterentwicklungen erscheinen. Diese sollen uns jedoch im weiteren Analyseverlauf nicht stören.

## 2. Datensatz Download und Import

Die Abbildung unten zeigt die hierarchische Ansicht eines **Enterprise Resource Planning (ERP) Systems**, wie beispielsweise SAP-ERP, das buchhalterische Vorgänge in Form von Journaleinträgen in Datenbanktabellen aufzeichnet. Im Kontext forensischer Datenanalysen können die in solchen Systemen erfassten bzw. gesammelten Daten wertvolle Hinweise auf mögliche Betrugsversuche enthalten.

Der synthetische **Forensic Accounting** Datensatz stellt einen Datenextrakt aus SAP Finance, einem der am meisten verbreitetsten ERP-Systeme, dar. Konkret umfasst der Datensatz die beiden grundlegenden Tabellen eines **SAP Finance (FI) Moduls**. Das SAP FI Modul umfasst sämtliche Geschäftsprozesse im Bereich des Finanz- und Rechnungswesens.

Dazu gehören unter anderem die **Debitoren- und Kreditorenbuchhaltung** sowie die **Haupt- und Nebenbuchhaltung**. Die über das Modul SAP FI abgebildeten Prozesse dienen somit der doppelten Buchführung, Erfassung von Belegen auf den erforderlichen Konten und der damit einhergehenden Gewinnermittlung für externe (Finanzamt) und interne Zwecke (Geschäftsführung).

<img align="center" style="max-width: 700px" src="./assets/sap_system.png">

Insgesamt umfasst der Forensic Accounting Datensatz **3.313 Belegeköpfe (Tabelle BKPF)** und **2.811 Belegsegmente (Tabelle BSEG)** des Geschäftsjahres 2016 eines anonymisierten Unternehmens. Die **Tabelle BKPF** enthält die allgemeinen Beleginformationen, z.B. die nachfolgenden Tabellenfelder:

>- `MANDT`: Client
>- `BUKRS`: Company Code
>- `BELNR`: Accounting Document Number
>- `GJAHR`: Fiscal Year
>- `BLART`: Document Type
>- `USNAM`: User Name
>- `TCODE`: Transaction Code
>- `BLDAT`: Document Date in Document
>- `CPUDT`: Posting Date Recorded
>- `BUDAT`: Posting Date in Document
>- ...

Weitere Informationen zu den BKPF Tabellenfeldern: https://www.leanx.eu/en/sap/table/bkpf.html. 

Die **Tabelle BSEG** enthält die detaillierten Beleginformationen auf Ebene der gebuchten Haupt- und Nebenbuchkonten, z.B. die nachfolgenden Tabellenfelder:

>- `MANDT`: Client
>- `BUKRS`: Company Code
>- `BELNR`: Accounting Document Number
>- `GJAHR`: Fiscal Year
>- `BUZEI`: Posting Line Item
>- `BSCHL`: Posting Key
>- `DMBTR`: Amount in Local Currency
>- `SHKZG`: Debit/Credit Indicator
>- `HKONT`: General Ledger Account
>- `SGTXT`: Item Text
>- `KUNNR`: Customer Identifier
>- `LIFNR`: Vendor Identifier
>- ...

Weitere Informationen zu den BSEG Tabellenfeldern: https://www.leanx.eu/en/sap/table/bseg.html. 

Die aktuelle Version des Datensatzes wurde am 20. April 2018 durch Michael Schermann an der Technischen Universität München als Teil des damaligen **'White-Collar Hacking Contests** veröffentlicht. Weitere Einzelheiten zu diesem Datensatz können der nachfolgenden Veröffentlichung entnommen werden: *Schermann, Michael, and Scott R. Boss. "The White-Collar Hacking Contest: A Novel Approach to Teach Forensic Investigations in a Digital World." (2014).* Der Datensatz selbst kann über die nachfolgende **GitHub-Webseite** bezogen werden: https://github.com/mschermann/forensic_accounting.

### 2.1. Download des Forensic Accounting Datensatzes

Lassen Sie uns nun die beiden grundlegenden Tabellen der **Belegköpfe (BKPF)** und **Belegsegmente (BSEG)** des Forensic Accounting Datensatzes herunterladen und lokal speichern. Hierzu ist es zunächst notwendig, den Pfad bzw. die Internetadressen der zu importierenden Tabellen zu definieren:

In [ ]:
bkpf_table_url = 'https://raw.githubusercontent.com/mschermann/forensic_accounting/master/BKPF.csv'
bseg_table_url = 'https://raw.githubusercontent.com/mschermann/forensic_accounting/master/BSEG.csv'

Im Sinne guter forensischer Praxis und den damit verbundenen Dokumentationszwecken erzeugen wir auch einen **Zeitstempel des Datendownloads**. Die Erstellung des Zeitstempels erfolgt über die `utcnow` Anweisung ([Dokumentation](https://docs.python.org/3/library/datetime.html)) der `datetime` Bibliothek und Formatangabe des Zeitstempels:

In [ ]:
# definition des Zeitstempels des Datendownloads
timestamp = dt.datetime.utcnow().strftime('%Y-%m-%d_%H-%M-%S')

In einem nächsten Schritt öffnen wir eine Verbindung zu beiden zuvor definierten Internetadressen. Hierzu verwenden wir die Anweisung `request.urlopen` der bereits importierten `URLLib` Bibliothek:

In [ ]:
bkpf_request = urllib.request.urlopen(bkpf_table_url) # erstellen einer Verbindung zur BKPF URL
bseg_request = urllib.request.urlopen(bseg_table_url) # erstellen einer Verbindung zu BSEG URL

Im Anschluss werden die Daten beider Tabellen **unverändert** d.h. im sogenannten **Byte-Format** heruntergeladen:

In [ ]:
bkpf_data_raw = io.BytesIO(bkpf_request.read()) # einlesen der BKPF Daten
bseg_data_raw = io.BytesIO(bseg_request.read()) # einlesen der BSEG Daten

Abschliessend speichern wir eine Arbeitskopie beider Tabellen auf dem lokalen Dateisystem im CSV-Format. Das Speichern erfolgt innerhalb des bereits zuvor erstellten Ordners für zu archivierende **erhaltene Originaldaten** mit der Bezeichnung ***01_original_data***. Hierzu verwenden wir die Anweisung `write_bytes` der importierten `Pathlib` Bibliothek.

In [ ]:
# definition des Dateinamens der BKPF Originaldaten
bkpf_original_file_name = timestamp + '_bkpf_file_original.csv'

# definition Dateipfades der BKPF Originaldaten
bkpf_path_original = os.path.join(original_data_dir, bkpf_original_file_name)

# speichern der BKPF Originaldaten
pathlib.Path(bkpf_path_original).write_bytes(bkpf_data_raw.getbuffer())

# definition des Dateinamens der BSEG Originaldaten
bseg_original_file_name = timestamp + '_bseg_file_original.csv'

# definition Dateipfades der BSEG Originaldaten
bseg_path_original = os.path.join(original_data_dir, bseg_original_file_name)

# speicher der BSEG Originaldaten
pathlib.Path(bseg_path_original).write_bytes(bseg_data_raw.getbuffer());

Nachfolgend speichern wir die unveränderten Daten beider Tabellen erneut auf dem lokalen Dateisystem im CSV-Format. Der zweite Speichervorgang erfolgt innerhalb des bereits zuvor erstellten Ordners für zu **analysierende Originaldaten** mit dem Namen ***02_analysis_data*** ab:

In [ ]:
# definition des Dateinamens der BKPF Analysedaten
bkpf_analysis_file_name = timestamp + '_bkpf_file_analysis.csv'

# definition Dateipfades der BKPF Originaldaten
bkpf_path_analysis = os.path.join(analysis_data_dir, bkpf_analysis_file_name)

# speichern der BKPF Originaldaten
pathlib.Path(bkpf_path_analysis).write_bytes(bkpf_data_raw.getbuffer())

# definition des Dateinamens der BSEG Originaldaten
bseg_analysis_file_name = timestamp + '_bseg_file_analysis.csv'

# definition Dateipfades der BSEG Originaldaten
bseg_path_analysis = os.path.join(analysis_data_dir, bseg_analysis_file_name)

# speicher der BSEG Originaldaten
pathlib.Path(bseg_path_analysis).write_bytes(bseg_data_raw.getbuffer());

### 2.2. Import des Forensic Accounting Datensatzes

In einem nächsten Schritt möchten wir beide Tabellen jeweils als `Pandas` DataFrame in das Notebook importieren. Hierzu ist es notwendig, die beiden im CSV-Format vorliegenden Dateien über die `read_csv` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)) der `Pandas` Bibliothek einzulesen:

In [ ]:
bkpf_data = pd.read_csv(bkpf_path_analysis, sep=',', index_col=0, thousands=',') # einlesen der BKPF Tabelle 
bseg_data = pd.read_csv(bseg_path_analysis, sep=',', index_col=0, thousands=',') # einlesen der BSEG Tabelle 

Lassen Sie uns nun die **5 ersten Zeilen** der Tabelle **Belegköpfe (BKPF)** anschauen um zu überprüfen, dass die Daten grundsätzlich im richtigen Format d.h. als `Pandas` DataFrame importiert wurden. Hierzu verwenden wir die `head` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html)) der `Pandas` Bibliothek:

In [ ]:
bkpf_data.head(5)

Über die `tail` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.tail.html)) der `Pandas` Bibliothek ist es zudem z.B. möglich auch die **5 letzten Zeilen** der Tabelle **Belegköpfe (BKPF)** auszugeben. Auch hier lohnt es sich wieder zu überprüfen, dass die Daten grundsätzlich im richtigen Format d.h. als DataFrame importiert wurden. Drüber hinaus kann geprüft werden, ob im Rahmen des Imports sog. Versatzfehler aufgetreten sind:

In [ ]:
bkpf_data.tail(5)

## 3. Strukturelle Datenvalidierung

Im Rahmen der **strukturellen Datenvalidierung** wird überprüft, ob technische Diskrepanzen zwischen der erwarteten Datenpopulation und der tatsächlich erhalten Datenpopulation existieren, z.B. bedingt durch fehlende Datensätze oder Formatierungsfehler. Das Hauptziel der strukturellen Datenvalidierung besteht darin, die technische Vollständigkeit und Integrität der erhaltenen Daten sicherzustellen.

Nachfolgend möchten wir grundlegende Schritte der strukturellen Datanvalidierung durchführen, diese umfassen: die Validierung der Anzahl Belegzeilen (**Abschnitt 3.1**) und die Validierung der Merkmalsdatentypen (**Abschnitt 3.2**).

### 3.1. Validierung der Belegzeilen

Nachfolgend soll nun vereinfacht eine strukturelle Validierung der Belegköpfe und Belegsegmente durchgeführt werden. In einem ersten Schritt validieren wir die Vollständigkeit der erhaltenen Daten. Hierzu ermitteln wir die Anzahl der Datenzeilen und Datenspalten und gleichen diese Information mit der erwarteten Zeilen- und Spaltenanzahl ab. Der Abgleich erfolgt unter Verwendung der `shape` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html)) der `Pandas`Bibliothek. 

Lassen Sie uns zunächst die Tabelle **Belegköpfe (BKPF)** auswerten:

In [ ]:
bkpf_data.shape

Lassen Sie uns die Tabelle **Belegsegmente (BSEG)** auswerten:

In [ ]:
bseg_data.shape

Die eingelesene Tabelle BKPF umfasst insgesamt **3.313 Zeilen** und **113 Spalten**, die Tabelle BSEG umfasst **2.811 Zeilen** und **336 Spalten**. Diese Informationen gilt es nun mit der erhaltenen Dokumentation über die Daten abzugleichen.

### 3.2. Validierung der Datentypen

In einem zweiten Schritt sollen die Datentypen der einzelnen Datenmerkmale validiert werden. In Python und im Besonderen der `Pandas` Bibliothek werden grds. die nachfolgenden Datentypen unterschieden:

>- `int64` - bezeichnet numerische Werte natürlicher bzw. ganzer Zahlen.
>- `float64` - bezeichnet numerische Werte reeller bzw. fliesskomma Zahlen.
>- `object` - bezeichnet alphanumerische Werte bzw. Textfelder.
>- `bool` - bezeichnet binäre 'Wahr' und 'Falsch' Wertfelder.
>- `datetime64` - bezeichnet Datum und Zeit Wertfelder. 

Eine Übersicht und detailierte Beschreibung der verschiedenen Datentypen in `Pandas` findet sich in der nachfolgenden Referenz ([Pandas Datentypen](https://pbpython.com/pandas_dtypes.html)). Die Datentypen der einzelnen Spalten eines `DataFrames` können anhand der `dtypes` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html)) der `Pandas` Bibliothek ermittelt werden.


Lassen Sie uns auch diesmal wieder mit Tabelle **Belegköpfe (BKPF)** beginnen:

In [ ]:
bkpf_data.dtypes

Lassen Sie uns wieder die Tabelle **Belegsegmente (BSEG)** auswerten:

In [ ]:
bseg_data.dtypes

Auch die gewonnenen Informationen zu den verschiedenen Datentypen gilt es wieder mit der erhaltenen Datendokumentation abzugleichen. 

Beispielsweise wird auf Grundlage der Validierung deutlich, dass der Datentyp des Feldes Belegnummer (Tabellenfeld: BELNR) in beiden Tabellen einen **unterschiedlichen Datentyp** aufweist. In der Tabelle BKPF den Typ `int64` (natürliche Zahlen) und in der Tabelle BSEG den Typ `float64` (reelle Zahlen). Um diese Dateninkonsistenz zu beseitigen wandeln wir den Datentyp des Tabellenfelds BELNR in der Tabelle BSEG in `int64` (reelle Zahlen) um. Die Umwandlung erfolgt unter Verwendung der `astype` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html)) der `Pandas` Bibliothek:

In [ ]:
bseg_data['BELNR'] = bseg_data['BELNR'].astype(np.int64)

Lassen Sie uns nun erneut die Datentypen der Tabelle **Belegsegmente (BSEG)** auswerten:

In [ ]:
bseg_data.dtypes

## 4. Datensatz Aufbereitung

Der Schritt der **Datenaufbereitung** stellt einen elementaren Bestandteil im Rahmen der Durchführung forensischer Datenanalysen dar. Unter Datenaufbereitung wird die **Bereinigung und Transformation von Rohdaten vor der Verarbeitung und Auswertung verstanden**. Dies ermöglicht eine effiziente Analyse und kann die Anzahl möglicher Fehler und Ungenauigkeiten während der Analysen reduzieren. 

Nachfolgend möchten wir grundlegende Schritte der forensischen Datenaufbereitung durchführen, diese umfassen: die Erstellung eines Identifikationsmerkmals (**Abschnitt 4.1**), die Auswahl analyserelevanter Datenmerkmale (**Abschnitt 4.2**), die Aufbereitung ausgewählter Datenmerkmale (**Abschnitt 4.3**) sowie die Verknüpfung verschiedener Datenmerkmale (**Abschnitt 4.4**).

### 4.1. Erstellung eines Identifikationsmerkmals

Ein eindeutiges **Identifikationsmerkmal** wird innerhalb der forensischen Datenanalyse zur eindeutigen Kennzeichnung einzelner Datensätze verwendet. Diese Kennzeichnung ist notwendig, um einzelne Datensätze im weiteren Analysevorgehen eindeutig identifizieren zu können. Eine solches Identifikationsmerkmal besteht häufig aus einer **verketteten Folge von vorhandenen Datenmerkmalen**. Die Datenmerkmale werden so ausgewählt werden, dass die Verkettung für jede Zeile des Datensatzes ein eindeutiges Identifizierungsmerkmal darstellt.

Im Kontext von SAP-Buchhaltungsbelegen hat sich die Verkettung der nachfolgenden Merkmale zur Erstellung eindeutiger Identifikationsmerkmale bewährt. Für die SAP-Tabelle der Belegköpfe (BKPF) wird das Identifikationsmerkmal oftmals anhand der vier Merkmale **Mandant (MANDT)**, **Buchungskreis (BUKRS)**, **Geschäftsjahr (GJAHR)** und **Belegnummer (BELNR)** erstellt. Nachfolgend erzeugen wir nun ein solches Identifikationsmerkmal in einer neuen Spalte der Tabelle BKPF mit der Bezeichnung **KEY** durch die Verkettung der Merkmale:

In [ ]:
bkpf_data['KEY'] = bkpf_data['MANDT'].astype(str) + '_' + bkpf_data['BUKRS'].astype(str) + '_' + bkpf_data['GJAHR'].astype(str) + '_' + bkpf_data['BELNR'].astype(str)

Lassen Sie uns nun die korrekte Anlage des Identifikationsmerkmals anhand der **5 ersten Zeilen** der Tabelle **Belegköpfe (BKPF)** überprüfen:

In [ ]:
bkpf_data.head(5)

Für die SAP-Tabelle der Belegsegmente (BSEG) wird das Identifikationsmerkmal auch anhand der vier Merkmale **Mandant (MANDT)**, **Buchungskreis (BUKRS)**, **Geschäftsjahr (GJAHR)** und **Belegnummer (BELNR)** erstellt. Nachfolgend erzeugen wir wieder nun ein solches Identifikationsmerkmal in einer neuen Spalte innerhalb der Tabelle BSEG mit der Bezeichnung **KEY** durch die Verkettung der Merkmale:

In [ ]:
bseg_data['KEY'] = bseg_data['MANDT'].astype(str) + '_' + bseg_data['BUKRS'].astype(str) + '_' + bseg_data['GJAHR'].astype(str) + '_' + bseg_data['BELNR'].astype(str)

Lassen Sie uns nun auch wieder die korrekte Anlage des Identifikationsmerkmals anhand der **5 ersten Zeilen** der Tabelle **Belegsegmente (BSEG)** überprüfen:

In [ ]:
bseg_data.head(5)

Da wir nun in beiden Tabellen jeder Datenzeile ein eindeutiges Identifikationsmerkmal zugewiesen haben, können wir in einem nächsten Schritt mit der strukturellen Validierung der Buchungsdaten fortfahren.

### 4.2. Auswahl von Datenmerkmalen

Im Rahmen einer forensischen Datenanalyse ist es gängige Praxis, die Datenmerkmale (Tabellenfelder) der zu analysierenden Daten auf die für eine Analyse notwendigen Merkmale zu reduzieren. Die Reduktion geschieht oftmals z.B. aus Gründen des Datenschutzes und der damit geforderten Datensparsamkeit jedoch oftmals aus Gründen der Analyseeffizienz. 

Lassen Sie uns hierzu zunächst die Datenmerkmale der beiden Tabellen Belegköpfe (BKPF) und Belegsegmente (BSEG) festlegen, welche für die nachfolgenden Analysen notwendig sind. Die **relevanten Datenmerkmale** werden nachfolgend für in Form einer Python `Liste` definiert:

In [ ]:
bkpf_fields = ['KEY', 'MANDT', 'BUKRS', 'BELNR', 'GJAHR', 'BLART', 'USNAM', 'TCODE', 'BLDAT', 'CPUDT', 'BUDAT', 'BKTXT', 'WAERS']
bseg_fields = ['KEY', 'MANDT', 'BUKRS', 'BELNR', 'GJAHR', 'BUZEI', 'BSCHL', 'DMBTR', 'WRBTR', 'SHKZG', 'HKONT', 'SGTXT', 'KUNNR', 'LIFNR']

In einem nächsten Schritt werden die Tabellen um die nicht benötigten Datenmerkmale bereinigt: 

In [ ]:
bkpf_data_sel = bkpf_data[bkpf_fields]
bseg_data_sel = bseg_data[bseg_fields]

Lassen Sie uns nun das Filterergebnis anhand der **5 ersten Zeilen** der Tabelle **Belegköpfe (BKPF)** überprüfen:

In [ ]:
bkpf_data_sel.head(5)

Lassen Sie uns das Filterergebnis auch anhand der **5 ersten Zeilen** der Tabelle **Belegsegmente (BSEG)** überprüfen:

In [ ]:
bseg_data_sel.head(5)

### 4.3. Aufbereitung von Datenmerkmalen

Ob es sich bei einer Buchungszeile um eine Soll- oder Habenzeile handelt, wird in SAP FI anhand **des sog. Soll- und Haben-Kennzeichens der Belegsegmente festgelegt** (Tabelle BSEG, Tabellenfeld: SHKZG). Die Ausprägung 'S' des Soll- und Haben-Kennzeichens bezeichnet eine Soll-Position und die Ausprägung 'H' eine Haben-Position. Eine Aufbereitung der Betragsfelder um diese Information ist für nachfolgende Validierungs- bzw. Analyseschritten oftmals hilfreich, um aussagekräftiger Ergebnisse zu erzielen. 

Um eine solche Aufbereitung vorzunehmen, erstellen wir zunächst eine Kopie der beiden Betragsmerkmale:

In [ ]:
# kopieren der Betragsfelder DMBTR und WRBTR
bseg_data_sel['DMBTR_VZ'] = bseg_data_sel['DMBTR']
bseg_data_sel['WRBTR_VZ'] = bseg_data_sel['WRBTR']

In einem nächsten Schritt werden die kopierten Betragsmerkmale mit einem entsprechenden Vorzeichen versehen. Hierbei ist es eine übliche Konvention, dass Sollzeilen ein **positives Vorzeichen** und Habenzeilen ein **negatives Vorzeichen** aufweisen. Die Anreicherung beider Betragsmerkmale erfolgt durch die Verwendung der `iloc` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html)) der `Pandas` Bibliothek:

In [ ]:
# anreichern des Betragsfeldes DMBTR_VZ mit Vorzeichen
bseg_data_sel.loc[bseg_data_sel['SHKZG'] == 'S', 'DMBTR_VZ'] *= 1.0
bseg_data_sel.loc[bseg_data_sel['SHKZG'] == 'H', 'DMBTR_VZ'] *= -1.0

# anreichern des Betragsfeldes WRBTR_VZ mit Vorzeichen
bseg_data_sel.loc[bseg_data_sel['SHKZG'] == 'S', 'WRBTR_VZ'] *= 1.0
bseg_data_sel.loc[bseg_data_sel['SHKZG'] == 'H', 'WRBTR_VZ'] *= -1.0

### 4.4. Verknüpfung von Datenmerkmalen

In einem letzten Schritt der Datenaufbereitung führen wir unterschiedliche Datenquellen in einer **gemeinsamen Datengrundlage zusammen**. Für die verschiedenen Zielsetzungen nachfolgender Analysen sollen die Belegkopfdaten und Belegsegmentdaten miteinander verknüpft werden. Das nachfolgende Schaubild zeigt beispielhaft die Beziehung beider Datengrundgesamtheiten über die vier Schlüsselmerkmale **Mandant (MANDT)**, **Buchungskreis (BUKRS)**, **Geschäftsjahr (GJAHR)** und **Belegnummer (BELNR)**.

<img align="center" style="max-width: 700px" src="./assets/document_linkage.png">

Das Erstellen einer solchen Verknüpfung ermöglicht die Analyse relationaler Beziehungen zwischen Belegköpfen und Belegsegmenten. Die Verknüpfung beider Beleginformationen erfolgt durch die Verwendung der `merge` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)) der `Pandas` Bibliothek:

In [ ]:
bkpf_bseg_sel_merged = bkpf_data_sel.merge(bseg_data_sel, on='KEY', how='inner', suffixes=('_bkpf', '_bseg'))

Lassen Sie uns nun das Ergebnis der Verknüpfung beider Tabellen anhand der **10 ersten Zeilen** der Verknüpfungstabelle überprüfen:

In [ ]:
bkpf_bseg_sel_merged.head(5)

Über die Anzahl der Einträge innerhalb der Verknüpfungstabelle erhalten wir eine Auskunft darüber, wie viele Verknüpfungen zwischen beiden Tabellen erstellt werden konnten. Lassen Sie uns deshalb die Anzahl der Zeilen innerhalb der Verknüpfungstabelle prüfen:

In [ ]:
bkpf_bseg_sel_merged.shape

Die Anzahl von 2.810 gefundenen Verknüpfungen veranschaulicht, dass **nicht für jeden Belegkopf zumindest ein zugehöriges Belegsegment gefunden wurde**. 

Diese Beobachtung kann im Kontext von SAP FI zunächst einen Hinweis auf eine etwaige Inkonsistenz der Daten darstellen. Jedoch können auch valide Gründe für eine solche Beobachtung existieren, so werden innerhalb von SAP FI durch sog. **Belegklammern** die Ursprungsbeleg und Ausgleichsbeleg zusammenfassen. Solche Belegklammern werden in Form von Belegköpfen innerhalb der Tabelle BKPF erfasst. Die gewonnenen Erkenntnisse gilt es jedoch zu plausibilisieren.

## 5. Semantische Datenvalidierung

Im Rahmen der **semantische Datenvalidierung** wird überprüft, ob die empfangenen Daten semantische Inkonsistenzen zwischen den erwarteten Datenausprägungen und den tatsächlich Datenausprägungen aufweisen, z.B. bedingt durch unvollständige oder fehlerhafte Datenmerkmale. Das Hauptziel der semantischen Datenvalidierung besteht darin, die semantische Vollständigkeit und Integrität der erhaltenen Daten sicherzustellen.

Nachfolgend möchten wir grundlegende Schritte der semantischen Datenvalidierung durchführen, diese umfassen: die Validierung ausgewählter Buchungseigenschaften (**Abschnitt 5.1**), die semantische Validierung kategorischer Datenmerkmale (**Abschnitt 5.2**), numerischer Datenmerkmale (**Abschnitt 5.3**) und kombinierter Datenmerkmale (**Abschnitt 5.4**).

### 5.1. Semantische Validierung der Buchungseigenschaften

Im Kontext der doppelten Buchführung saldieren sich die Soll- und Habenpositionen einer Buchung grundsätzlich zu null. Abweichungen hiervon können Hinweise auf fehlerhafte Buchungen oder einen unvollständigen Datensatz darstellen. Diese grundlegende Buchungseigenschaft wird nun in einem weiteren Schritt validiert. 

Hierzu werden die einzelnen **Belegsegmente (BSEG)** pro Buchung anhand des zuvor erstellten eindeutigen Identifikationsmerkmals aufsaldiert. Das Saldieren erfolgt unter Verwendung der `groupby` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)) der `Pandas` Bibliothek:

In [ ]:
posting_saldo = bkpf_bseg_sel_merged.groupby(by=['KEY'], dropna=False)['DMBTR_VZ'].agg(['count', 'sum'])

Lassen Sie uns nun den Soll und Haben Saldo der einzelnen Buchungen prüfen:

In [ ]:
posting_saldo

In einem weiteren Schritt filtern wir nun gezielt Buchungen, die einen positiven oder negativen Saldo aufweisen:

In [ ]:
posting_saldo_incomplete = posting_saldo[np.round(posting_saldo['sum']) != 0.00]

Lassen Sie uns nun die gefilterten Belegsegmente auch innerhalb des Notebooks prüfen:

In [ ]:
posting_saldo_incomplete

Auf Grundlage dieses Schrittes der Validierung konnten **insgesamt 19 Buchungen ermittelt werden, die unvollständig erscheinen**. Lassen Sie uns die einzelnen Belegsegmente dieser Buchungen innerhalb des Notebooks extrahieren. Zum Filtern dieser Belegköpfe kann auf die Anweisung `isin` der `Pandas` Bibliothek zurückgegriffen werden:

In [ ]:
bseg_posting_saldo_incomplete = bseg_data_sel[bseg_data_sel['KEY'].isin(posting_saldo_incomplete.index)]

Lassen Sie uns nun das Filterergebnis anhand der **5 ersten Zeilen** der extrahierten **Belegsegmente (BKPF)** überprüfen:

In [ ]:
bseg_posting_saldo_incomplete.head(5)

Abschliessend möchten wir nun die gefilterten Belegsegmente der Buchungen in eine **externe Excel-Datei** für nachgelagerte substanzielle Prüfungshandlungen zu extrahieren. Zu Dokumentationszwecken erzeugen wir zunächst wieder einen **Zeitstempel des Datenexports**. Die Erstellung des Zeitstempels erfolgt über die `utcnow` Anweisung ([Dokumentation](https://docs.python.org/3/library/datetime.html)) der `datetime` Bibliothek und Formatangabe des Zeitstempels:

In [ ]:
timestamp = dt.datetime.utcnow().strftime('%Y-%m-%d_%H-%M-%S')

Für den tatsächlichen Export der entsprechenden Belegsegmente kann auf die Anweisung `to_excel` der `Pandas` Bibliothek zurückgegriffen werden. Der zweite Speichervorgang erfolgt innerhalb des bereits zuvor erstellten Ordners für **Validierungsergebnisse** mit dem Namen ***03_validation_results***:

In [ ]:
# definition des Dateinamens
file_name = str(timestamp) + '-VA_001_incomplete_postings.xlsx'

# definition des Dateipfades
file_directory = os.path.join(validation_data_dir, file_name)

# extraktion der Daten nach Excel
bseg_posting_saldo_incomplete.to_excel(file_directory, header=True, index=False, sheet_name='Incomplete_KEY', encoding='utf-8')

### 5.2. Semantische Validierung kategorischer Datenmerkmale

In einem weiteren Schritt validieren wir die semantische Integrität der erhaltenen **kategorischen Datenfelder**. Hierzu wird die Anzahl unterschiedlicher Merkmalsausprägungen ermittelt. Die nachfolgende Validierung erfolgt beispielhaft anhand der in Tabelle **Belegköpfe (BKPF)** enthaltenen Datenmerkmale unter Verwendung der `nunique` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html)) der`Pandas` Bibliothek:

In [ ]:
bkpf_data_sel.nunique()

Auf Grundlage der Analyse wird deutlich, dass die Belegköpfe zu acht unterschiedlichen Belegarten (Tabellenfeld: BLART) korrespondieren. Darüber hinaus ist z.B. auffällig, dass keine der Buchungen einen Belegkopftext (Tabellenfeld: BKTXT) aufweist. Die gewonnenen Erkenntnisse gilt es im Nachgang mit einer entsprechenden Erwartungshaltung abzugleichen bzw. zu plausibilisieren.

Um die Ausprägungen der Belegarten im Detail zu analysieren, kann u.a. die `value_counts` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html)) der `Pandas` Bibliothek verwendet werden. Diese ermöglicht es beispielsweise die Anzahl Belegköpfe pro Belegart zu ermitteln:

In [ ]:
bkpf_data_sel['BLART'].value_counts()

Die Validierung zeigt, dass die überwiegende Mehrheit der Belegköpfe Warenausgängen (BLART: WA) entsprechen. Darüber hinaus enthalten die Belegköpfe ein paar sehr selten verwendete Belegarten. Auch diese Erkenntnisse gilt es im Nachgang wieder zu plausibilisieren.

Für eine aussagekräftige Dokumentation der gewonnenen Informationen kann es sinnvoll sein, die Verteilung der Belgarten z.B. anhand eines **Histograms** zu visualisieren. Zur Erstellung eines solchen Diagramms kann wieder auf die Funktionalität der `Matplotlib` Bibliothek zurückgegriffen werden:

In [ ]:
# initialisieren des Diagramms
fig, ax = plt.subplots()

# erstellen des Histograms der Belegartenverteilung
bkpf_data_sel['BLART'].value_counts().plot(ax=ax, kind='bar', color='cornflowerblue', alpha=0.8)

# hinzufügen der Achsenbeschriftungen
plt.ylabel('Posting Count', fontsize=12)
plt.xlabel('Document Type (BLART)', fontsize=12)

# hinzufügen des Titels
plt.title('Distribution of Document Type', fontsize=12);

Lassen Sie uns nun die insgesamt neun Belegköpfe extrahieren, die Buchungen betreffen, welche über selten verwendete der Belegarten gebucht werden. Zum Filtern dieser Belegköpfe kann wieder auf die `isin` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html)) der `Pandas` Bibliothek zurückgegriffen werden:

In [ ]:
bkpf_unusual_blart = bkpf_data_sel[bkpf_data_sel['BLART'].isin(['RV' , 'DZ', 'WL'])]

Lassen Sie uns nun die gefilterten Belegköpfe auch innerhalb des Notebooks prüfen:

In [ ]:
bkpf_unusual_blart

Abschliessend möchten wir zudem die gefilterten Belegköpfe in eine **externe Excel-Datei** auf dem lokalen Dateisystem exportieren. Dies kann beispielsweise für nachgelagerte substanzielle Prüfungshandlungen sinnvoll erscheinen. 

Zu Dokumentationszwecken erzeugen wir zunächst wieder einen **Zeitstempel des Datenexports**. Die Erstellung des Zeitstempels erfolgt über die `utcnow` Anweisung ([Dokumentation](https://docs.python.org/3/library/datetime.html)) der `datetime` Bibliothek und Formatangabe des Zeitstempels:

In [ ]:
timestamp = dt.datetime.utcnow().strftime('%Y-%m-%d_%H-%M-%S')

Für den tatsächlichen Export der entsprechenden Belegköpfe kann auf die `to_excel` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html)) der `Pandas` Bibliothek zurückgegriffen werden. Der zweite Speichervorgang erfolgt innerhalb des bereits zuvor erstellten Ordners für **Validierungsergebnisse** mit dem Namen ***03_validation_results***:

In [ ]:
# definition des Dateinamens
file_name = str(timestamp) + '-VA_002_unusal_document_types.xlsx'

# definition des Dateipfades
file_directory = os.path.join(validation_data_dir, file_name)

# extraktion der Daten nach Excel
bkpf_unusual_blart.to_excel(file_directory, header=True, index=False, sheet_name='Unusual_BLART', encoding='utf-8')

### 5.3. Semantische Validierung numerischer Datenmerkmale

In einem zweiten Schritt validieren wir die semantische Integrität der erhaltenen **numerischen Datenfelder**. Hierzu werden verschiedene Verteilungsstatistiken berechnet. Die nachfolgende Validierung erfolgt beispielhaft anhand der Tabelle **Belegsegmente (BSEG)** und der Datenmerkmale Betrag in Hauswährung (Tabellenfeld: DMBTR) und Betrag in Fremdwährung (Tabellenfeld: WRBTR) unter Verwendung der `describe` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)) der `Pandas` Bibliothek:

In [ ]:
bseg_data_sel[['DMBTR', 'WRBTR']].describe()

Auf Grundlage der Validierung wird deutlich, dass einzelne Belegsegmente ungewöhnlich hohe Buchungsbeträge in Hauswährung als auch in Fremdwährung aufweisen. Die Beträge weichen in beiden Fällen signifikant um mehrere Standardabweichungen vom Mittelwert der jeweilig erfassten Buchungsbeträge ab. Die gewonnenen Informationen gilt es im Anschluss mit einer entsprechenden Erwartungshaltung abzugleichen bzw. zu plausibilisieren.

Für eine aussagekräftige Dokumentation kann es wieder sinnvoll sein, die Verteilung der Beträge z.B. anhand eines **Boxplots** zu visualisieren. Zur Erstellung eines solchen Diagramms kann wieder auf die Funktionalität der `Matplotlib` Bibliothek zurückgegriffen werden:

In [ ]:
# initialisieren des Diagramms
fig, ax = plt.subplots()

# erstellen des Boxplots der Betragsverteilungen
bseg_data_sel.boxplot(column=['DMBTR', 'WRBTR'])

# hinzufügen der Achsenbeschriftungen
plt.ylabel('Posting Amount', fontsize=12)
plt.xlabel('Posting Amount Field', fontsize=12)

# hinzufügen des Titels
plt.title('Distribution of Posting Amounts', fontsize=12);

Lassen Sie uns nun die Belegsegmente extrahieren, welche einen ungewöhnlich hohen Buchungsbetrag aufweisen. Hierzu berechnen wir zunächst den **Mittelwert** und die **Standardabweichung** des Betrags in Hauswährung (Tabellenfeld: DMBTR). Zur Berechnung dieser Statistiken kann auf die Anweisungen `mean` ([Dokumentation](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)) und `std` ([Dokumentation](https://numpy.org/doc/stable/reference/generated/numpy.std.html)) der `NumPy` Bibliothek zurückgegriffen werden:

In [ ]:
# berechnung des Mittelwertes
dmbtr_mean = np.mean(bseg_data_sel['DMBTR'])

# berechnung der Standardabweichung
dmbtr_std = np.std(bseg_data_sel['DMBTR'])

In einem nächsten Schritt möchten wir nun Belegsegmente, deren Betrag in Hauswährung (Tabellenfeld: DMBTR) grösser oder gleich $n=3$ positive Standardabweichungen vom Mittelwert abweichen. Zum Filtern dieser Belegköpfe kann auf die gewöhnliche Filterfunktionalität der `Pandas` Bibliothek zurückgegriffen werden:

In [ ]:
# setzen der Anzahl positiver Standardabweichungen
n = 3

# filtern von Belegsegmenten ungewöhnlicher hoher Buchungsbeträge
bseg_unusual_amount = bseg_data_sel[bseg_data_sel['DMBTR'] >= (n * dmbtr_std) + dmbtr_mean]

Lassen Sie uns nun die gefilterten Belegsegmente auch innerhalb des Notebooks prüfen:

In [ ]:
bseg_unusual_amount

Wir möchten nun die gefilterten Belegsegmente wieder in eine **externe Excel-Datei** auf dem lokalen Dateisystem exportieren. Dies kann beispielsweise für nachgelagerte substanzielle Prüfungshandlungen oder sinnvoll erscheinen. 

Zu Dokumentationszwecken erzeugen wir zunächst wieder einen **Zeitstempel des Datenexports**. Die Erstellung des Zeitstempels erfolgt über die `utcnow` Anweisung ([Dokumentation](https://docs.python.org/3/library/datetime.html)) der `datetime` Bibliothek und Formatangabe des Zeitstempels:

In [ ]:
timestamp = dt.datetime.utcnow().strftime('%Y-%m-%d_%H-%M-%S')

Für den tatsächlichen Export der entsprechenden Belegsegmente kann auf die `to_excel` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html)) der `Pandas` Bibliothek zurückgegriffen werden:

In [ ]:
# definition des Dateinamens
file_name = str(timestamp) + '-VA_003_unusal_posting_amounts.xlsx'

# definition des Dateipfades
file_directory = os.path.join(validation_data_dir, file_name)

# extraktion der Daten nach Excel
bseg_unusual_amount.to_excel(file_directory, header=True, index=False, sheet_name='Unusual_DMBTR', encoding='utf-8')

### 5.4. Semantische Validierung kombinierter Datenmerkmale

In einem dritten Schritt validieren wir die semantische Datenintegrität der Kombination unterschiedlicher Datenmerkmale. Hierzu berechnen wir auch wieder verschiedene Verteilungsstatistiken. Die nachfolgende Validierung erfolgt beispielhaft anhand der Tabelle **Belegsegmente (BSEG)** und der Datenmerkmale Hauptbuchkonto (Tabellenfeld: HKONT) und Buchungschlüssel (Tabellenfeld: BSCHL) unter Verwendung der `pivot_table` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html)) der `Pandas` Bibliothek:

In [ ]:
pd.pivot_table(bseg_data_sel, values='KEY', index=['HKONT'], columns=['BSCHL'], aggfunc=np.size, fill_value=0)

Die Validierung zeigt diverse seltene bzw. 'ungewöhnliche' Kombinationen Haupttbuchkonto-Buchungsschlüssel-Kombinationen. Beispielsweise wurde das **Hauptbuchkonto 741000** lediglich ein einziges mal über den **Buchungschlüssel 50** bebucht. Auch hier gilt es die gewonnenen Informationen im Anschluss zu plausibilisieren.

Für eine aussagekräftige Dokumentation kann es wieder sinnvoll sein, die Verteilung der Kombination aus Hauptbuchkonto (Tabellenfeld: HKONT) und Buchungsschlüssel (Tabellenfeld: BSCHL) z.B. anhand einer **Heatmap** zu visualisieren. Zur Erstellung eines solchen Diagramms kann auf die Funktionalität der `Matplotlib` Bibliothek in Kombination mit der `Seaborn` Bibliothek zurückgegriffen werden:

In [ ]:
# initialisieren des Diagramms
fig, ax = plt.subplots(figsize=(7,7))

# erstellen der Pivot Tabelle aus Hauptbuchkonto und Buchungsschlüssel
pivot = pd.pivot_table(bseg_data_sel, values='KEY', index=['HKONT'], columns=['BSCHL'], aggfunc=np.size, fill_value=0)

# visualisierung der Pivot Tabelle als Seaborn Heatmap
sns.heatmap(pivot, square=True, annot=True, cbar=False, cmap='Blues', fmt='g')

# hinzufügen der Achsenbeschriftungen
plt.ylabel('General-Ledger Account (HKONT)', fontsize=12)
plt.xlabel('Posting Key (BSCHL)', fontsize=12)

# hinzufügen des Titels
plt.title('Pivot General-Ledger Account vs. Posting Key', fontsize=12);

Lassen Sie uns nun die insgesamt 6 Belegsegmente extrahieren, die Buchungen betreffen, welche über eine **selten verwendete Kombination** von Hauptbuchkonto (Tabellenfeld: HKONT) und Buchungsschlüssel (Tabellenfeld: BSCHL) aufweisen. Zum Filtern dieser Belegköpfe kann auf die multiple Filterfunktionalität der `Pandas` Bibliothek zurückgegriffen werden:

In [ ]:
# filtern ungewöhnlicher Hauptbuchkonto und Buchungsschlüssel Kombinationen
bseg_unsual_account_key_1 = bseg_data_sel[(bseg_data_sel['HKONT'] == 760000) & (bseg_data_sel['BSCHL'] == 40)]
bseg_unsual_account_key_2 = bseg_data_sel[(bseg_data_sel['HKONT'] == 741000) & (bseg_data_sel['BSCHL'] == 81)] 
bseg_unsual_account_key_3 = bseg_data_sel[(bseg_data_sel['HKONT'] == 740000) & (bseg_data_sel['BSCHL'] == 50)] 
bseg_unsual_account_key_4 = bseg_data_sel[(bseg_data_sel['HKONT'] == 210000) & (bseg_data_sel['BSCHL'] == 50)]
bseg_unsual_account_key_5 = bseg_data_sel[(bseg_data_sel['HKONT'] == 210000) & (bseg_data_sel['BSCHL'] == 81)] 

# zusammenführen der Teilergebnisse
bseg_unsual_account_key = pd.concat([bseg_unsual_account_key_1, bseg_unsual_account_key_2, bseg_unsual_account_key_3, bseg_unsual_account_key_4, bseg_unsual_account_key_5], axis=0)

Lassen Sie uns nun die gefilterten Belegsegmente auch innerhalb des Notebooks prüfen:

In [ ]:
bseg_unsual_account_key

Wir möchten nun die gefilterten Belegsegmente wieder in eine **externe Excel-Datei** auf dem lokalen Dateisystem exportieren. Dies kann beispielsweise für nachgelagerte substanzielle Prüfungshandlungen oder sinnvoll erscheinen. 

Zu Dokumentationszwecken erzeugen wir zunächst wieder einen **Zeitstempel des Datenexports** über die `utcnow` Anweisung ([Dokumentation](https://docs.python.org/3/library/datetime.html)) der `datetime` Bibliothek und Formatangabe des Zeitstempels:

In [ ]:
timestamp = dt.datetime.utcnow().strftime('%Y-%m-%d_%H-%M-%S')

Für den tatsächlichen Export der entsprechenden Belegsegmente kann wieder auf die `to_excel` Anweisung ([Dokumentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html)) der `Pandas` Bibliothek zurückgegriffen werden:

In [ ]:
# definition des Dateinamens
file_name = str(timestamp) + '-VA_004_unusal_accounts_and_posting_keys.xlsx'

# definition des Dateipfades
file_directory = os.path.join(validation_data_dir, file_name)

# extraktion der Daten nach Excel
bseg_unsual_account_key.to_excel(file_directory, header=True, index=False, sheet_name='Unusual_HKONT_BSCHL', encoding="utf-8")

## Lab Aufgaben:

Im Ihr wissen zu vertiefen empfehlen wir, die nachfolgenden Übungen zu bearbeiten:

**1. Validierung der verwendeten Benutzerkennungen.**

> Validieren Sie die innerhalb der Belegköpfe **verwendeten Benutzerkennungen** (Tabelle: BKPF, Tabellenfeld: USNAM). Erstellen Sie hierzu eine Übersicht der im Datensatz enthaltenen Benutzerkennungen. Ermitteln Sie für jede Benutzerkennung die Anzahl erfasster Belegköpfe unter Verwendung der `value_counts` Anweisung der `Pandas` Bibliothek.  Extrahieren Sie abschliessend die Belegköpfe der Benutzerkennungen, die **weniger als 10 Belegköpfe erfassten** in eine gesonderte Excel-Datei.

In [ ]:
# ***************************************************
# Sie können Ihre Lösung an dieser Stelle einfügen
# ***************************************************

**2. Validierung der Kombination verwendeter Lieferanten und Buchungstexte.**

> Validieren Sie die Kombination der innerhalb der Belegsegmente **verwendeten Lieferanten** (Tabelle: BSEG, Tabellenfeld: LIFNR) in Kombination mit den **erfassten Buchungstexten** (Tabelle: BSEG, Tabellenfeld: SGTXT). Erstellen Sie hierzu eine Pivot Übersicht der im Datensatz enthaltenen Kombinationen aus Lieferanten und Buchungstexten. Ermitteln Sie für jede Kombination die **Anzahl erfasster Belegsegmente** unter Verwendung der `pivot_table` Anweisung der `Pandas` Bibliothek. Extrahieren Sie abschliessend die Belegsegmente die **den Buchungstext 'Consulting' aufweisen** in eine gesonderte Excel-Datei.

In [ ]:
# ***************************************************
# Sie können Ihre Lösung an dieser Stelle einfügen
# ***************************************************

**3. Validierung der Kombination verwendeter Lieferanten und Benutzerkennungen.**

> Validieren Sie die Kombination der innerhalb der Belegsegmente **verwendeten Lieferanten** (Tabelle: BSEG, Tabellenfeld: LIFNR) und **Benutzerkennungen** (Tabelle: BKPF, Tabellenfeld: USNAM). Erstellen Sie hierzu eine Pivot Übersicht der im Datensatz enthaltenen Kombinationen aus Lieferanten und Benutzerkennungen. Ermitteln Sie für jede Kombination **die Anzahl, das Betragsvolumen und das durchschnittliche Betragsvolumen erfasster Belegsegmente** (Tabelle: BSEG, Tabellenfeld: DMBTR) unter Verwendung der `group_by` Anweisung der `Pandas` Bibliothek. Extrahieren Sie abschliessend die Belegsegmente, welche die **Benutzerkennung 'GBI-010'** und den **Lieferant '125114'** aufweisen in eine gesonderte Excel-Datei.

In [ ]:
# ***************************************************
# Sie können Ihre Lösung an dieser Stelle einfügen
# ***************************************************

## Lab Zusammenfassung:

Dieses zweite Lab Notebook umfasst eine schrittweise Einführung in die grundlegenden Konzepte der Datenvalidierung im Kontext forensischer Datenanalysen. Dabei wurden insbesondere die Teilschritte **(1) Datenimport**,  **(2) Datenaufbereitung** und **(3) Datenvalidierung** behandelt. Die vorgestellten Code Beispiele und die Übungen können als Ausgangspunkt für komplexere und Ihre massgeschneiderten Analysen dienen.